## One Body Integrals of Qiskit and PySCF

In [1]:
import numpy as np
import pylab
from pyscf import gto, scf, dft
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

Numpy 1.16 has memory leak bug  https://github.com/numpy/numpy/issues/13808
It is recommended to downgrade to numpy 1.15 or older


Calcs here in units of:
Angstrom

Calcs in Szabo & Ostlund in units of:
AU (Bohr)

Conversion:
1 AU = 0.5291772083

### Define Molecule
The PySCFDriver function relies on functions in PySCF to calculate the one-body integrals.  When they come out of the driver function, they are not in the right basis yet for use in the second-quantized Hamiltonian. 

In [2]:
molecule = 'H .0 .0 -0.3704; H .0 .0 0.3704'
driver = PySCFDriver(molecule, basis='sto3g')
qmolecule = driver.run()

### One-Body Integrals in PySCF
`mo_coeff` comes directly from the PySCF function as shown here:

In [13]:
test_mol = gto.Mole()
test_mol.atom = [['H', (0.7408, 0., 0.)], ['H', (0.,0.,0.)]]
test = scf.hf.RHF(test_mol)
test.kernel()
print(test.mo_coeff)
#print(test.get_hcore())
type(test)

converged SCF energy = -1.11671690917341


Initialize <pyscf.gto.mole.Mole object at 0x7f2b6a2182e8> in <pyscf.scf.hf.RHF object at 0x7f2b6a1bc748>


-1.1167169091734095

[[ 0.54892884  1.21152003]
 [ 0.54892884 -1.21152003]]


pyscf.scf.hf.RHF

#### What Qiming said: 
Dear Catherine,

mo_coeff is short for molecular orbital (MO) coefficients. The one body Hamiltonian in atomic orbital basis can be obtained

`h_ao = test.get_hcore()`

In most cases, you would need the one-body Hamiltonian in MO basis

`h_mo = test.mo_coeff.T.dot(test.get_hcore()).dot(test.mo_coeff)`

The Hamiltonian from Qiskit is likely a representation of the one-body Hamiltonian in N-body basis. It needs some transformation or direct product of based on h_mo depending the convention of the N-body basis of Qiskit.

Best,
Qiming Sun


In [14]:
h_ao = test.get_hcore()
h_ao

h_mo = test.mo_coeff.T.dot(test.get_hcore()).dot(test.mo_coeff)
h_mo

array([[-1.12044021, -0.95843642],
       [-0.95843642, -1.12044021]])

array([[-1.25282613e+00, -4.20539999e-17],
       [ 9.89338352e-18, -4.75572084e-01]])

### One-Body Integrals in Qiskit
Qiskit takes it further by converting the `mo_coeff` of PySCF into the so-called 'spin-orbital basis'. According to folks on Qiskit, it is this basis that the integrals must be in to become the input for the one and two-body integrals of the second quantized Hamiltonian. We see in the cell below, that at this point, the Qiskit output is exactly the same as the PySCF output. 

In [5]:
#dir(qmolecule)
qmolecule.mo_coeff
#qmolecule.mo_coeff_b
#qmolecule.mo_onee_ints_b

array([[ 0.54892884,  1.21152003],
       [ 0.54892884, -1.21152003]])

In what follows below, he use the code from the `integrals.py` script in Qiskit, taken from Github: https://github.com/Qiskit/qiskit-aqua/blob/master/qiskit/chemistry/drivers/pyscfd/integrals.py

In [15]:
isinstance(qmolecule.mo_coeff, tuple)
len(qmolecule.mo_coeff.shape) 
## so therefore these definitons are made (see integrals.py for if/then statement)
mo_coeff = qmolecule.mo_coeff
mo_coeff_b = None

False

2

In [16]:
hij = scf.RHF(test_mol).get_hcore() ## qmolecule does not have 'get_hcore' attribute
hij

array([[-1.12044021, -0.95843642],
       [-0.95843642, -1.12044021]])

#### `mohij` are one body integrals in molecular orbital basis

In [17]:
mohij = np.dot(np.dot(mo_coeff.T, hij), mo_coeff)
mohij



array([[-1.25282613e+00, -4.20539999e-17],
       [ 9.89338352e-18, -4.75572084e-01]])

#### These are inputs to `onee_to_spin`
which are called by the `one_body_integrals` function

The `onee_to_spin` function: 
"Convert one-body MO integrals to spin orbital basis.
Takes one body integrals in molecular orbital basis and returns integrals in spin orbitals ready for use as coefficients to one body terms 2nd quantized Hamiltonian."

In [18]:
mo_onee_ints = mohij
mo_onee_ints_b = None

In [19]:
one_body_integrals = qmolecule.onee_to_spin(mo_onee_ints, mo_onee_ints_b)

In [20]:
one_body_integrals
qmolecule.one_body_integrals

array([[-1.25282613,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.47557208,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.25282613,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.47557208]])

array([[-1.25282613,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.47557208,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.25282613,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.47557208]])

### OK so those go into 2nd quantized Hamiltonian...does pyscf not calculate them tho? Why does Qiskit have to do through all this to get them into this 'spin orbital basis'?

In [21]:
def onee_to_spin(mohij, mohij_b=None, threshold=1E-12):
        """Convert one-body MO integrals to spin orbital basis

        Takes one body integrals in molecular orbital basis and returns
        integrals in spin orbitals ready for use as coefficients to
        one body terms 2nd quantized Hamiltonian.

        Args:
            mohij (numpy.ndarray): One body orbitals in molecular basis (Alpha)
            mohij_b (numpy.ndarray): One body orbitals in molecular basis (Beta)
            threshold (float): Threshold value for assignments
        Returns:
            numpy.ndarray: One body integrals in spin orbitals
        """
        if mohij_b is None:
            mohij_b = mohij

        # The number of spin orbitals is twice the number of orbitals
        norbs = mohij.shape[0]
        nspin_orbs = 2*norbs

        # One electron terms
        moh1_qubit = numpy.zeros([nspin_orbs, nspin_orbs])
        for p in range(nspin_orbs):  # pylint: disable=invalid-name
            for q in range(nspin_orbs):
                spinp = int(p/norbs)
                spinq = int(q/norbs)
                if spinp % 2 != spinq % 2:
                    continue
                ints = mohij if spinp == 0 else mohij_b
                orbp = int(p % norbs)
                orbq = int(q % norbs)
                if abs(ints[orbp, orbq]) > threshold:
                    moh1_qubit[p, q] = ints[orbp, orbq]

        return moh1_qubit

In [22]:
from pyscf import gto, scf

## define molecule, H2 at 0.2 Angstrom
test_mol = gto.Mole()
test_mol.atom = [['H', (0.7408, 0., 0.)], ['H', (0.,0.,0.)]]
test = scf.hf.RHF(test_mol)
test.kernel()

## construct 'one-body-integrals' for input to 2nd quantized Hamiltonian
mo_coeff = qmolecule.mo_coeff
hij = scf.RHF(test_mol).get_hcore()
mohij = np.dot(np.dot(mo_coeff.T, hij), mo_coeff)
mohij_b = None
one_body_integrals = qmolecule.onee_to_spin(mohij, mohij_b)

## integrals from PySCF
mo_coeff
## integrals from Qiskit
one_body_integrals

converged SCF energy = -1.11671690917341


Initialize <pyscf.gto.mole.Mole object at 0x7f2b6a1bca58> in <pyscf.scf.hf.RHF object at 0x7f2b6a1bc6a0>


-1.1167169091734095

array([[ 0.54892884,  1.21152003],
       [ 0.54892884, -1.21152003]])

array([[-1.25282613,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.47557208,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.25282613,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.47557208]])